In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils import data

import torchvision
from matplotlib import pyplot as plt
from utils import *

batch_size = 128

#step1:load the dataset
train_loader = data.DataLoader(torchvision.datasets.FashionMNIST("./mnist_data", train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(), 
                               torchvision.transforms.Normalize(
                                (0.1307,),(0.3081,))
                                ])), 
        batch_size=batch_size, shuffle=True)

test_loader = data.DataLoader(torchvision.datasets.FashionMNIST("./mnist_data", train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(), 
                               torchvision.transforms.Normalize(
                                (0.1307,),(0.3081,))
                                ])), 
        batch_size=batch_size, shuffle=False)

x, y = next(iter(train_loader))
#print(x.shape,y.shape)
#plt_image(x, y, 'image sample')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #xw+b
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
learning_rate = 0.01
optimizer = optim.SGD(net.parameters(),lr=learning_rate, momentum=0.9)
train_loss = []

for epoch in range(3):
    for batch_idx, (x, y) in enumerate(train_loader):
        #x(batch_size,1,28,28)->(batch_size,28*28)
        x = x.view(x.size(0),28*28)
        #->[b,10]
        out = net(x)
        #->[b,10]
        y_onehot = one_hot(y)
        #loss = mse(out, y_onehot)
        loss = F.mse_loss(out, y_onehot)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        train_loss.append(loss.item())
        if batch_idx % 50 ==0:
            print(epoch, batch_idx, loss.item())

plot_curve(train_loss)
        
        



In [ ]:
total_correct = 0
for x,y in test_loader:
    x = x.view(x.size(0),28*28)
    out = net(x)
    #out:[b,10]->pred:[b]
    pred = out.argmax(dim=1)
    correct = pred.eq(y).sum().float().item()
    # print(pred)
    # print(y)
    total_correct += correct

total_num = len(test_loader.dataset)
acc = total_correct / total_num
print("test acc:", acc)
#保存模型
torch.save(net.state_dict(), 'model.pth')
    